reference : https://stackoverflow.com/questions/16992713/translate-every-element-in-numpy-array-according-to-key

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
pd.options.display.max_rows = 6

In [2]:
df = sns.load_dataset('exercise').iloc[:,[1,2,5]]
df.head()

,id,diet,kind
0,1,low fat,rest
1,1,low fat,rest
2,1,low fat,rest
3,2,low fat,rest
4,2,low fat,rest


In [5]:
df['kind'].cat.categories

Index(['rest', 'walking', 'running'], dtype='object')

- [일반적인 데이터 mapping](#일반적인-데이터-mapping)

- [데이터 조건에 따른 mapping](#데이터-조건에-따른-mapping)

## 일반적인 데이터 mapping

.replace 함수 : 지정된 값만 찾아 변경

In [6]:
map_dict= {'rest': 'first' , 'walking': 'second', 'running': 'third'}

In [42]:
df['kind'].replace(map_dict)

0       first
1       first
2       first
       ...   
87    running
88    running
89    running
Name: kind, Length: 90, dtype: object

### .map 함수 활용

replace 함수도 있지만 데이터가 많을수록 .map을 활용하는 것이 더 빠르다.

In [11]:
df['kind'].map(map_dict)

0     first
1     first
2     first
      ...  
87      NaN
88      NaN
89      NaN
Name: kind, Length: 90, dtype: object

단, dict에 지정하지 않은 값들은 Nan으로 처리된다. // numpy 방법은 해결 가능

In [38]:
map_dict= {'rest': 'first' , 'walking': 'second'}

In [10]:
df['kind'].map(map_dict)

0     first
1     first
2     first
      ...  
87      NaN
88      NaN
89      NaN
Name: kind, Length: 90, dtype: object

### numpy 활용

In [43]:
np.vectorize(map_dict.get)(df['kind'])

array(['first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
       'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
       'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None',
       'None', 'None', 'None', 'None', 'None', 'None', 'None'],
      dtype='<U6')

dict에 지정하지 않은 값들은 자신의 값 그대로 활용 가능

In [29]:
np.vectorize(map_dict.get)(df['kind'], df['kind'])

array(['first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'first', 'first', 'first', 'first', 'first',
       'first', 'first', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'second', 'second', 'second', 'second', 'second',
       'second', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'running', 'running',
       'running', 'running', 'running', 'running', 'running', 'running',
       'runni

참고

In [41]:
%timeit df['kind'].map(map_dict)

611 µs ± 5.63 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [34]:
%timeit np.vectorize(map_dict.get)(df['kind'])

165 µs ± 550 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 데이터 조건에 따른 mapping

### np.where 활용

np.where( 조건식, True 결과, False 결과 )

In [50]:
df = sns.load_dataset('exercise').iloc[:,[1,2,3]]
df.head()

,id,diet,pulse
0,1,low fat,85
1,1,low fat,85
2,1,low fat,88
3,2,low fat,90
4,2,low fat,92


### 다중 조건에는 np.select 혹은 pd.cut or np.digitize 활용

#### pd.cut

In [54]:
df['result1'] = pd.cut(df['pulse'],bins=[-np.inf,70,90,np.inf], labels=['Bad','SoSo','Good'])

-np.inf < Bad <=70

70 < Soso <= 90

90 < Good < np.inf

#### np.digitize

right=True / False 조건을 pd.cut과 맞춰야 한다.

In [51]:
result = np.array(['Bad','SoSo','Good'])
bins = np.digitize(df['pulse'].values, bins=[70, 90], right=True) # right 조건 체크

In [52]:
df['result2'] =  bins.choose(result)
# bins.choose(result) == result[bins]

In [55]:
df

,id,diet,pulse,result2,result1
0,1,low fat,85,SoSo,SoSo
1,1,low fat,85,SoSo,SoSo
2,1,low fat,88,SoSo,SoSo
...,...,...,...,...,...
87,30,no fat,99,Good,Good
88,30,no fat,111,Good,Good
89,30,no fat,150,Good,Good


#### np.select

조건식 결과가 중복되지 않게 조절해야 함

In [4]:
condlist = [df['pulse'].values<70, df['pulse'].values>90] # 조건식 형태로 입력
choicelist = ['Bad', 'Good']

In [5]:
df['result'] = np.select(condlist, choicelist, default='SoSo')

In [6]:
df

,id,diet,pulse,result
0,1,low fat,85,SoSo
1,1,low fat,85,SoSo
2,1,low fat,88,SoSo
...,...,...,...,...
87,30,no fat,99,Good
88,30,no fat,111,Good
89,30,no fat,150,Good
